# Introduction

The National Longitudinal Survey of Youth 1997-2011 dataset is one of the most important databases available to social scientists working with US data. 

It allows scientists to look at the determinants of earnings as well as educational attainment and has incredible relevance for government policy. It can also shed light on politically sensitive issues like how different educational attainment and salaries are for people of different ethnicity, sex, and other factors. When we have a better understanding how these variables affect education and earnings we can also formulate more suitable government policies. 

<center><img src=https://i.imgur.com/cxBpQ3I.png height=400></center>


### Upgrade Plotly

In [ ]:
#%pip install --upgrade plotly

###  Import Statements


In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt


# Machine learning stuff
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split

# Evaluating predictions
from sklearn.metrics import mean_squared_error



## Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

# Load the Data



In [ ]:
df = pd.read_csv('NLSY97_subset.csv')

### Understand the Dataset

Have a look at the file entitled `NLSY97_Variable_Names_and_Descriptions.csv`. 

---------------------------

    :Key Variables:  
      1. S           Years of schooling (highest grade completed as of 2011)
      2. EXP         Total out-of-school work experience (years) as of the 2011 interview.
      3. EARNINGS    Current hourly earnings in $ reported at the 2011 interview

# RQ1: What variables predict earnings?
# RQ1a: What variables positively predict earnings?
# RQ1b: What variables negatively predict earnings?

# Preliminary Data Exploration 🔎

**Challenge**

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
df

## Data Cleaning - Check for Missing Values and Duplicates

Find and remove any duplicate rows.

In [ ]:
df[df.duplicated().values]

In [ ]:
# Remove duplicated rows
df = df.drop_duplicates(ignore_index=True)

## Descriptive Statistics

In [ ]:
print(f"Ratio of remaining IDs to number of predictors: {len(df)/(len(df.columns)-1)}")

Considering the above, will the number of predictors still work for more advanced prediction methods?

In [ ]:
df.describe()[1:]

In [ ]:
df.info()

In [ ]:
df.columns

## Considering there were 97 features, I had to narrow down what 1997(birth) - 2004 (elementary) predictors would likely predict 2011 earnings. 

In [ ]:
numerical_feat =[
    "ID", # key for matching
    'EARNINGS', # outcome
    'S', # years of schooling
    'EXP', # out of school work experience
    'BYEAR', # Year of birth
    'AGE', # Age at 2011 (will likely correlate with BYEAR)
    'HHINC97', # Gross household income
    'POVRAT97', # Ratio of Poverty level
    
    # Parental Monitoring (scale of 0 low, to 16 high)
    'PRMONM', 'PRMONF',
    
    # ASVAB battery scores
    'ASVABAR', 'ASVABWK', 'ASVABPC', 'ASVABMK', 'ASVABNO',
    'ASVABCS', 'ASVABC', 'ASVABC4', 'VERBAL', 'ASVABMV',
    
    # height and weight at 2004
    'HEIGHT', 'WEIGHT04',
    
    # Family background
    'SF', 'SM', 'SFR', 'SMR', 'SIBLINGS',
]

categorical_feat = [
    "ID", # key for matching
    'FEMALE', 
    'MALE',
    # Household structure 1997
    'HHBMBF', 'HHBMOF', 'HHOMBF',
    'HHBMONLY', 'HHBFONLY', 'HHOTHER',
    
    # Household location 1997
    'MSA97NO', 'MSA97NCC', 'MSA97CC',
    'MSA97NK', 'REG97NE', 'REG97NC',
    'REG97S', 'REG97W', 'RS97RURL', 
    'RS97URBN', 'RS97UNKN',
    
    # Ethnicity
    'ETHBLACK', 'ETHHISP', 'ETHWHITE',

    # Highest educational qualification
    'EDUCPROF', 'EDUCPHD',
    'EDUCMAST', 'EDUCBA', 
    'EDUCAA', 'EDUCHSD', 
    'EDUCGED', 'EDUCDO',
    
    # Faith:
    'FAITHN', 'FAITHP', 'FAITHC', 'FAITHJ', 'FAITHO','FAITHM',
    
     # Parenting style (0 or 1)
    'PRMSTYUN', 'PRMSTYPE', 'PRMSTYAN', 'PRMSTYAE',
    'PRFSTYUN', 'PRFSTYPE', 'PRFSTYAN', 'PRFSTYAE',
    

]

not_included =[
    # marital status at 2011
    'SINGLE', 'MARRIED',
    'COHABIT', 'OTHSING',
    
    # weight at 2011
    'WEIGHT11'
    
    # work related vars at 2011
    'JOBS', 'HOURS', 'TENURE', 'COLLBARG',
    
    # Category of employment at 2011
    'CATGOV', 'CATPRI', 'CATNPO', 'CATMIS','CATSE',
    
    # Living in 2011
    'URBAN', 'REGNE', 'REGNC', 'REGW', 'REGS',
    'MSA11NO', 'MSA11NCC', 'MSA11CC', 'MSA11NK', 'MSA11NIC'
]

## Visualise the Features

In [ ]:
num_df = df[numerical_feat]
num_df

In [ ]:
%%script False
for i in range(1, len(num_df.columns)):
    fig = px.histogram(num_df[numerical_feat[i]], title = f"{numerical_feat[i]}")
    fig.show()

## My Analysis of graphs

I will replace NAs with the mean/median after the next section

## Now to look at the cateogical features

In [ ]:
cat_df = df[categorical_feat]
cat_df[:3]

In [ ]:
# select only rows with presence of NAs
cat_df.isna().sum()[cat_df.isna().sum()>0]
missing_cat = cat_df.isna().sum()[cat_df.isna().sum()>0].index
missing_cat

easy to clean. On inspection, NA values are typically 0

In [ ]:
# fill NAs with 0
cat_df = cat_df.fillna(0)

## Handing missing data from numerical features

In [ ]:
# select columns with NAs
num_df.isna().sum()[num_df.isna().sum()>0]
missing_num = num_df.isna().sum()[num_df.isna().sum()>0].index
missing_num

# see columns with data
num_df[missing_num][:3]

### Columns with Missing numerical values
- HHINC97: Gross household income, $, in year prior to 1997 interview
- POVRAT97: Ratio of household income to poverty level, 1997
- PRMONM: Monitoring by mother
- PRMONF: Monitoring by father
- SFR: Years of schooling of residential Father
- SMR: Years of schooling of residential Mother

I need to see the range of each to determine the necessary actions.

In [ ]:
num_df[missing_num].describe()

### Actions required: Imputation
- HHINC97: mean values
- POVRAT97: mean values
- PRMONM: median values
- PRMONF: median values
- SFR: median values
- SMR: median values

In [ ]:
# HHINC97 mean imputation
num_df['HHINC97'] = num_df['HHINC97'].fillna(num_df['HHINC97'].mean())

# POVRAT97 mean imputation
num_df['POVRAT97'] = num_df['POVRAT97'].fillna(num_df['POVRAT97'].mean())

# lets code the next one...
features = ["PRMONM","PRMONF","SFR","SMR"]

for i in features:
    num_df[i] = num_df[i].fillna(num_df[i].median())

In [ ]:
num_df[missing_num].isna().sum()

### NA filled, we can merge the data
At this point in time, I can do one more step to do standardising of the values.

But maybe for my next submission?

## Merging the data

In [ ]:
df_new = num_df.merge(cat_df, on="ID")

In [ ]:
df_new = df_new.drop("ID", axis = 1)


In [ ]:
df_new[:3]

I learnt from previous projects to check after merging...

# Split Training & Test Dataset

We *can't* use all the entries in our dataset to train our model. Keep 30% of the data for later as a testing dataset (out-of-sample data).  

In [ ]:
earnings = df_new["EARNINGS"]
df_new = df_new.drop("EARNINGS", axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_new, earnings,
                                                    test_size = .3, 
                                                    random_state = 1047)

### Checking the outputs to see if they are okay

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

# Simple Linear Regression

Only use the years of schooling to predict earnings. Use sklearn to run the regression on the training dataset. How high is the r-squared for the regression on the training data? 

In [ ]:
S = pd.DataFrame(X_train["S"])


In [ ]:
%%time
LR = LinearRegression()
LR.fit(S, y_train)

### Evaluate the Coefficients of the Model

Here we do a sense check on our regression coefficients. The first thing to look for is if the coefficients have the expected sign (positive or negative). 

Interpret the regression. How many extra dollars can one expect to earn for an additional year of schooling?

In [ ]:
print(f"For every one year of schooling, earnings increase by ${LR.coef_[0]:.2f} per hour.")

### Analyse the Estimated Values & Regression Residuals

How good our regression is also depends on the residuals - the difference between the model's predictions ( 𝑦̂ 𝑖 ) and the true values ( 𝑦𝑖 ) inside y_train. Do you see any patterns in the distribution of the residuals?

In [ ]:
S_score = LR.score(S, y_train)

In [ ]:
print(f"The number of years in education only accounts for {S_score*100:.2f}% of variance attributed to earnings.")

# Multivariable Regression

Now use both years of schooling and the years work experience to predict earnings. How high is the r-squared for the regression on the training data? 

In [ ]:
LR1 = LinearRegression()
LR1.fit(X_train[["S", "EXP"]], y_train)
LR1_score = LR1.score(X_train[["S", "EXP"]], y_train)

In [ ]:
LR1_score

In [ ]:
LR1.coef_

In [ ]:
print(f"The number of years in education and experience accounts for {LR1_score*100:.2f}% of variance attributed to earnings.")

### Evaluate the Coefficients of the Model

In [ ]:
print(f"For every number of years in education, earnings increase by ${LR1.coef_[0]:.2f} per hour." )
print(f"For every number of years with work experience, earnings increase by ${LR1.coef_[1]:.2f} per hour." )

Note: there could be an interaction effect, where more years of work increase the effect of education on earnings.  

# Use Your Model to Make a Prediction

How much can someone with a bachelors degree (12 + 4) years of schooling and 5 years work experience expect to earn in 2011?

In [ ]:
test_pred = pd.DataFrame({"S":16, "EXP":5}, index=[1])
score = LR1.predict(test_pred)

In [ ]:
print(f"A person with 16 years of schooling and 5 years of work experience is likely to earn ${score[0]:.2f} per hour in 2011.")

# Experiment and Investigate Further

Which other features could you consider adding to further improve the regression to better predict earnings? 

## Creating a scoring metric

In [ ]:
def RMSE(y_pred, y_true):
    score = mean_squared_error(y_pred,y_true)
    score = np.sqrt(score)
    
    return score

## BASELINE MODEL WITH ALL FEATURES

In [ ]:
%%time
LR2 = LinearRegression()
LR2.fit(X_train, y_train)
y_pred = LR2.predict(X_test)

In [ ]:
LR2_score = RMSE(y_pred, y_test)
LR2_score

In [ ]:
print(f"The root mean square error score for including ALL variables is {LR2_score:.4f}. That's quite high!")

In [ ]:
# comparing with only 2 predictors

y_pred_2 = LR1.predict(X_test[["S", "EXP"]])
LR1_score = RMSE(y_pred_2, y_test)

In [ ]:
print(f"The root mean square error score for including ALL variables is {LR1_score:.4f} (lower is better). That difference is negligible.")

<h1> <span style='background:yellow'> There are many ways to reduce the RMSE </span> </h1>

<ol>
<li> Feature selection (not all features contribute to the prediction)
<li> Scaling and standardisation (IMPT!)
<li> Use a different model. There are many algorithms to predict continuous variables. 

    
- SGD Regressor (scaling required)